In [ ]:
# %pip install faker
# %pip install sqlite
# %pip install pandas

In [1]:
import pandas as pd
import random
import sqlite3
import datetime

from faker import Faker
from datetime import timedelta

### **Variáveis**

**Users**
- user_id
- name
- age
- country

**Transactions**
- user_id
- transaction_id
- transaction_date
- transaction_state
- transaction_amount

## **Criando a base de dados:**


In [4]:
# Create connection to database
conn = sqlite3.connect('dbCompany')
cursor = conn.cursor()

# Create users table
cursor.execute(
                '''
                CREATE TABLE IF NOT EXISTS users (
                    user_id INTEGER PRIMARY KEY,
                    name TEXT NOT NULL,
                    age INTEGER NOT NULL,
                    country TEXT NOT NULL
                )
                '''
)

# Create transactions table
cursor.execute(
                '''
                CREATE TABLE IF NOT EXISTS transactions (
                    user_id INTEGER NOT NULL,
                    transaction_id INTEGER PRIMARY KEY,
                    transaction_date TEXT NOT NULL,
                    transaction_state TEXT NOT NULL,
                    transaction_amount REAL NOT NULL,
                    FOREIGN KEY (user_id) REFERENCES users (user_id)
                )
                '''
               )

conn.commit()
conn.close()

## **Gerando uma base de dados aleatória**

In [ ]:
faker_us = Faker('en-US')
faker_br = Faker('pt-BR')
faker_fr = Faker('fr-FR')


countries = ['USA', 'Brazil', 'France']
data = {
    'user_id': [],
    'name': [],
    'age': [],
    'country': []
}

for i in range(100):  # Generate 100 users
    country = random.choice(countries)
    # Select appropriate faker based on country
    if country == 'USA':
        faker = faker_us
    elif country == 'Brazil':
        faker = faker_br
    else:
        faker = faker_fr
    data['user_id'].append(i + 1)
    data['name'].append(faker.name())
    data['age'].append(random.randint(18, 65))
    data['country'].append(country)

# Create transactions dataframe
transaction_states = ['INITIATED', 'SUCCESS', 'FRAUD', 'CANCELLED']
transactions = {
    'transaction_id': [],
    'user_id': [],
    'transaction_date': [], 
    'transaction_state': [],
    'transaction_amount': []
}

for i in range(1000):  # Generate 500 transactions
    transactions['transaction_id'].append(i + 1)
    transactions['user_id'].append(random.randint(1, 100))
    # Generate random timestamp between 2022-2023
    random_date = datetime.datetime(random.randint(2022, 2024), 1, 1) + timedelta(
        days=random.randint(0, 729),
        hours=random.randint(0, 23),
        minutes=random.randint(0, 59),
        seconds=random.randint(0, 59)
    )
    transactions['transaction_date'].append(random_date.strftime('%Y-%m-%d %H:%M:%S'))
    transactions['transaction_state'].append(random.choice(transaction_states))
    transactions['transaction_amount'].append(round(random.uniform(10000, 100000), 2))

users_df = pd.DataFrame(data)
transactions_df = pd.DataFrame(transactions)


display(users_df)
display(transactions_df)

In [ ]:
# Create database connection and cursor
conn = sqlite3.connect('dbCompany')
cursor = conn.cursor()

users_data = list(users_df.itertuples(index=False, name=None))
transactions_data = list(transactions_df.itertuples(index=False, name=None))

# Inserindo o dataframe users_df na tabela users
cursor.executemany('INSERT INTO users (user_id, name, age, country) VALUES (?, ?, ?, ?)', users_data)

# Inserindo o dataframe transactions_df na tabela transactions
cursor.executemany(
                    '''
                    INSERT INTO transactions 
                    (transaction_id, user_id, transaction_date, transaction_state, transaction_amount) 
                    VALUES (?, ?, ?, ?, ?)
                    '''
                    , transactions_data
)

conn.commit()

print(f'Número de usuários inseridos: {len(users_data)}\nNúmero de transações inseridas: {len(transactions_data)}')

**A.** Qual é a idade média de usuários do sistema por país

In [ ]:
query = '''
        SELECT country AS Country, ROUND(COUNT(user_id),2) AS 'Média de idade' 
        FROM users
        GROUP BY country
        '''

df_A = pd.read_sql(query, conn)

display(df_A)

**B.** Qual é o país com a maior quantidade de dinheiro transacionado (considere só transações finalizadas com sucesso ou ``SUCCESS``)

In [ ]:
query = '''
        WITH TotalSuccess AS (
                SELECT user_id, SUM(transaction_amount) AS transacoes 
                FROM transactions 
                WHERE transaction_state = 'SUCCESS'
                GROUP BY user_id
                )

        SELECT u.country AS País, SUM(t.transacoes) AS "Soma das Transações"
        FROM TotalSuccess t
        LEFT JOIN users u
        ON u.user_id = t.user_id
        GROUP BY u.country
        '''

df_B = pd.read_sql(query, conn)

display(df_B)

**C.** Qual é o país com maior taxa de fraude em porcentagem respeito ao número de transações totais no país

In [ ]:
query = '''
        WITH
        Total AS (
            SELECT u.country, COUNT(*) as total_transacoes
            FROM transactions t
            LEFT JOIN users u
            ON t.user_id = u.user_id 
            GROUP BY u.country
        ),
        Fraudes AS (
            SELECT u.country, COUNT(*) as total_fraud
            FROM transactions t
            LEFT JOIN users u
            ON t.user_id = u.user_id
            WHERE t.transaction_state = 'FRAUD'
            GROUP BY u.country
        )
        
        SELECT 
            t.country AS País,
            ROUND(CAST(f.total_fraud AS FLOAT) / t.total_transacoes * 100, 2) as 'Taxa de Fraudes'
        FROM Total t
        LEFT JOIN Fraudes f
        ON t.country = f.country
        ORDER BY 'Taxa de Fraudes' DESC
        '''

# Execute the SQL query and store results in a pandas DataFrame
df_C = pd.read_sql(query, conn)

# Display the resulting DataFrame showing country fraud percentages
display(df_C)

**D.** Na mesma linha da pergunta anterior, responda qual é a faixa de idade de usuários
que mais cometem fraude (em percentagem).

Separe as faixas etárias em ``< 18 anos, 18-30 anos, 30 - 45 anos, 45 - 60 anos, 60 > anos``

Considerar o fato que um usuário pode ter executado várias transações, das quais poucas (ou muitas) podem ter sido fraude entre as demais.

**E.** Imagine que a camada executiva da empresa dona do sistema, precisa criar um Dashboard para monitorar o estado das transações nos últimos 3 dias.

Criar uma query SQL que calcule:

1. ``Número e dinheiro das transações não finalizadas``

2. ``Número e dinheiro de transações finalizadas com sucesso (SUCCESS)``

3. ``Número e dinheiro de transações canceladas (CANCELLED)``

4. ``Número e dinheiro de fraudes (FRAUD)``

Agrupado por país e nos 3 dias anteriores de quando o executivo da empresa consulte seu Dashboard.

In [98]:
conn.close()